In [1]:
import json
import excel2json
import jsonlines 
import os       
import openai 
import pandas as pd

In [2]:
# Setup the environment
# Load your API key from an environment variable named as OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

#### Excel -> Json -> load Json to python as dictionary -> convert to str -> make jsonl -> send to GPT3
file : Bill of Materials-AR403__LABEL.xlsx

In [13]:
#converting Excel to json file 
excel2json.convert_from_file(r'Labelled Data\Bill of Materials-AR403__LABEL.xlsx')

#loading json file as python dictionary
BOM_AR403 = open(r'Labelled Data\Sheet1.json',)
BOM_AR403_dict = json.load(BOM_AR403)

#Convert to str and put it in the GPT3 accessible file format : {"text":"...","metadata":"..."}
gpt3_format_BOM_AR403 = { "text" : str(BOM_AR403_dict), "metadata" : ""}

#Convert to jsonl for sending it to GPT3 File
with jsonlines.open(r'Labelled Data\gpt3_format_BOM_AR403.jsonl', 'w') as writer:
    writer.write(gpt3_format_BOM_AR403)

# create file
openai.File.create(file=open(r'Labelled Data\gpt3_format_BOM_AR403.jsonl'), purpose='answers')


Upload progress: 100%|███████████████████████████████████████████████████████████| 21.2k/21.2k [00:00<00:00, 40.3kit/s]


<File file id=file-Szgpq0tBGYM98mrsTPrMx5sB at 0x26ec5a52dc8> JSON: {
  "bytes": 20893,
  "created_at": 1618497587,
  "filename": "Labelled Data\\gpt3_format_BOM_AR403.jsonl",
  "id": "file-Szgpq0tBGYM98mrsTPrMx5sB",
  "object": "file",
  "purpose": "answers",
  "status": "uploaded",
  "status_details": null
}

In [17]:
#Check if the file is retrievable and error free
openai.File.retrieve('file-Szgpq0tBGYM98mrsTPrMx5sB')

<File file id=file-Szgpq0tBGYM98mrsTPrMx5sB at 0x26ec5d06f48> JSON: {
  "bytes": 20893,
  "created_at": 1618497587,
  "filename": "Labelled Data\\gpt3_format_BOM_AR403.jsonl",
  "id": "file-Szgpq0tBGYM98mrsTPrMx5sB",
  "object": "file",
  "purpose": "answers",
  "status": "processed",
  "status_details": null
}

#### status : processed (we are good to go...)

In [16]:
#Bring GPT3 into action
openai.Answer.create( 
    model="davinci", 
    question="What is the value of MPN where Designators is R1, R5, R12, R13, R20, R53, R55, R57 and Qty is  8.0", 
    file="file-Szgpq0tBGYM98mrsTPrMx5sB", 
    examples_context=" \"OrigExcelRowNumber\": 73.0, \"Designators\": \"IC12\", \"Qty\": 1.0, \"Unit\": \"Pieces\", \"DNP\": 0, \"CPN\": \"\", \"MPN\": \"TPS54360DDA\", \"Manufacturer\": \"TEXAS INSTRUMENTS\", \"isManufacturerFree\": 0, \"ResistorProperties\": \"\", \"CapacitorProperties\": \"\", \"OtherProperties\": \"60V ; 3.5 A\", \"Package\": \"HSOP-8\", \"Description\": \"60V Input, 3.5 A, Step Down DC-DC Converter\", \"Comment\": \"\"}\n{\"OrigExcelRowNumber\": 16.0, \"Designators\": \"R1, R5, R12, R13, R20, R53, R55, R57\", \"Qty\": 8.0, \"Unit\": \"Pieces\", \"DNP\": 0, \"CPN\": \"\", \"MPN\": \"\", \"Manufacturer\": \"\", \"isManufacturerFree\": 1, \"ResistorProperties\": \"1k\", \"CapacitorProperties\": \"\", \"OtherProperties\": \"\", \"Package\": \"0805\", \"Description\": \"Resistor\", \"Comment\": \"\"}\n{\"OrigExcelRowNumber\": 18.0, \"Designators\": \"C15, C16, C18, C19, C20, C34\", \"Qty\": 6.0, \"Unit\": \"Pieces\", \"DNP\": 0, \"CPN\": \"\", \"MPN\": \"\", \"Manufacturer\": \"\", \"isManufacturerFree\": 1, \"ResistorProperties\": \"\", \"CapacitorProperties\": \"1µF\", \"OtherProperties\": \"\", \"Package\": \"0805\", \"Description\": \"Capacitor\", \"Comment\": \"\"}\n{\"OrigExcelRowNumber\": 23.0, \"Designators\": \"XP2\", \"Qty\": 1.0, \"Unit\": \"Pieces\", \"DNP\": 0, \"CPN\": \"\", \"MPN\": \"\", \"Manufacturer\": \"\", \"isManufacturerFree\": 1, \"ResistorProperties\": \"\", \"CapacitorProperties\": \"\", \"OtherProperties\": \"2.5 MSF 04\", \"Package\": \"\", \"Description\": \"Connector\", \"Comment\":\"\" ", 
    examples=[["What is the value of MPN where Designators is IC12 and Qty is  1.0","TPS54360DDA"],
    ["What is the value of MPN where Designators is  XP2 and Qty is 1.0","No value found"]],
    max_tokens=10,
    stop=["\n", "<|endoftext|>"]
)

InvalidRequestError: The document at index 1 is 6304 tokens over the length limit of 1994. If you would like us to add a feature to auto-truncate server-side, let us know at support@openai.com.